In [1]:
from functions import *

2024-11-21 15:12:19.009470: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 15:12:19.029609: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 15:12:19.035681: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 15:12:19.050961: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-21 15:12:20.187153: W tensorflow/compiler/tf2

In [2]:
import matplotlib.pyplot as plt
import glob
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import geopandas as gpd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import glob
import time
from datetime import datetime
from collections import Counter
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model

## Load model

In [4]:
tf.get_logger().setLevel('ERROR') 
model_name = 'baseline_CNN'
model = load_model(f'../models/{model_name}.h5')

2024-11-21 15:12:25.784360: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10530 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1
2024-11-21 15:12:25.784995: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10532 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:81:00.0, compute capability: 6.1


## Load data

In [7]:
filt_alt = False 
W=16
split='random'
inputs=['lst','ndvi']
if W==128:
        data_folder = '../data/preprocessed/'
else:
    data_folder = f'../data/preprocessed/{W}x{W}/'
        
data = load_all_data(
source_folder='../data/external/shp/river_cells_oficial',
source_path=data_folder,
data_paths= inputs,
filter_altitude=filt_alt,
W=W,
time_split=True if split=='time' else False)

lst : (1487, 16, 16, 3)
ndvi : (1487, 16, 16)


In [10]:
total_data, total_times, data_targets, labels = data
cosine_months, sine_months, cos_to_month = get_months_vectorized(total_times['lst'])
lat, lon = get_lat_lon(labels)
discharge = get_discharge(labels, total_times['lst'])
additional_inputs = np.column_stack((cosine_months, sine_months, lat, lon, discharge))

global_ranges = {}
for inp in inputs:  
    all_images = total_data[inp]  
    global_min = np.min(all_images)
    global_max = np.max(all_images)
    global_ranges[inp] = (global_min, global_max)
    print(f"Variable: {inp}, Min: {global_min}, Max: {global_max}")

expanded_images = []
for inp in inputs:  
    all_images = total_data[inp]  
    min_val, max_val = global_ranges[inp]  
    normalized_images = normalize_min_max(all_images, min_val, max_val)
    if normalized_images.ndim == 3:  
        normalized_images = np.expand_dims(normalized_images, axis=-1)  
    expanded_images.append(normalized_images)
combined_input = np.concatenate(expanded_images, axis=-1)  
input_data = combined_input

train_index, validation_index, test_index = get_split_index('stratified', input_data, data_targets, labels, 1, filt_alt)
validation_input = input_data[validation_index, :] 
validation_target = data_targets[validation_index]
test_input = input_data[test_index, :]
test_target = data_targets[test_index]
train_input = input_data[train_index, :]
train_target = data_targets[train_index]

Variable: lst, Min: 0.0, Max: 255.0
Variable: ndvi, Min: -0.5326488018035889, Max: 0.8705131411552429
yes !!!!!!!!!!


# Gradients on test set

In [25]:
test_inp_labels={}
for i,l in enumerate(np.array(labels)[test_index]):
    if l not in test_inp_labels:
        test_inp_labels[l] = [test_input[i]]
    else:
        test_inp_labels[l].append(test_input[i])
test_dates = np.array(total_times['lst'])[test_index]

In [27]:
print(len(test_inp_labels.values()))
print(len(test_dates))

53
298


In [30]:
len(test_inp_labels['cell_76'])

9

In [31]:
from collections import Counter
imgs = Counter(labels)
imgs['cell_76']

21

In [34]:
c=0
for cell, test_images in test_inp_labels.items():
    for test_image in test_images:
        test_image_tensor = tf.convert_to_tensor(test_image, dtype=tf.float32)
        test_image_tensor = tf.expand_dims(test_image_tensor, axis=0)
        # Use GradientTape to track gradients
        with tf.GradientTape() as tape:
            tape.watch(test_image_tensor)  # Watch the tensor
            preds = model(test_image_tensor, training=False)  # Perform prediction
            target_output = preds[:, 0]  # Assuming target_output is scalar, adjust as needed
        
        # Calculate the gradients
        gradients = tape.gradient(target_output, test_image_tensor)
        
        folder_path = f'../plots/grad_cam/{model_name}/{cell}'
        os.makedirs(folder_path,exist_ok=True)
        date = test_dates[c]
        image_path = os.path.join(folder_path, f'image_{c}.png')
        impact_channel = 3
        save_grad_map(gradients, image_path, impact_channel)
        c+=1
    

In [42]:
grad_folder = f'../plots/grad_cam/{model_name}'

for cell in test_inp_labels.keys():
    cell_grad_folder = os.path.join(grad_folder,cell)
    # List to store gradient images
    grad_images = []

    for grad_image_name in os.listdir(cell_grad_folder):
        grad_image_path = os.path.join(cell_grad_folder, grad_image_name)
    
        # Load the image
        grad_image = Image.open(grad_image_path)
        print(grad_image.size)
    print()
    '''
    # Resize the image to the target shape
    grad_image_resized = grad_image.resize(target_shape)

    # Convert the resized image to a numpy array and append to the list
    grad_image_resized = np.array(grad_image_resized)
    grad_images.append(grad_image_resized)
    
    # Now convert the list to a NumPy array
    grad_images = np.array(grad_images)
    
    # Calculate the mean of the gradients
    mean_grad = np.mean(grad_images, axis=0)
    
    plt.imshow(mean_grad, cmap="bwr")
    plt.colorbar()
    plt.title("Mean Gradients")
    plt.show()'''
    

(1511, 1304)
(1511, 1304)
(1537, 1304)
(1511, 1304)
(1537, 1304)
(1511, 1304)
(1511, 1304)
(1537, 1304)
(1511, 1304)

(1537, 1304)
(1511, 1304)
(1537, 1304)
(1511, 1304)

(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)

(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1537, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1537, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1510, 1304)
(1511, 1304)

(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1537, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1537, 1304)
(1511, 1304)
(1537, 1304)
(1511, 1304)

(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1537, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511, 1304)

(1511, 1304)
(1511, 1304)
(1511, 1304)
(1511,

### Mean of gradients

In [9]:
gradients_folder = '../plots/grad_cam/'

gradient_images = [os.path.join(gradients_folder,file) for file in os.listdir(gradients_folder) if not os.path.isdir(file)]
print(gradient_images)

# Lista para almacenar las medias de los gradientes
mean_gradients = []

# Iterar sobre las imágenes de gradientes
for image_path in gradient_images:
    grad_map = plt.imread(image_path)  # Cargar la imagen de gradientes
    mean_grad = np.mean(grad_map)  # Calcular la media de los gradientes
    mean_gradients.append(mean_grad)

# Mostrar la media de los gradientes por imagen
print("Media de los gradientes para cada imagen:", mean_gradients)

['../plots/grad_cam/gradients_epoch_52_batch_2_channel_3.png', '../plots/grad_cam/gradients_epoch_29_batch_3_channel_3.png', '../plots/grad_cam/gradients_epoch_56_batch_3_channel_3.png', '../plots/grad_cam/gradients_epoch_13_batch_2_channel_3.png', '../plots/grad_cam/gradients_epoch_92_batch_1_channel_3.png', '../plots/grad_cam/gradients_epoch_79_batch_1_channel_3.png', '../plots/grad_cam/gradients_epoch_65_batch_2_channel_3.png', '../plots/grad_cam/gradients_epoch_98_batch_2_channel_3.png', '../plots/grad_cam/gradients_epoch_70_batch_3_channel_3.png', '../plots/grad_cam/gradients_epoch_23_batch_1_channel_3.png', '../plots/grad_cam/gradients_epoch_18_batch_2_channel_3.png', '../plots/grad_cam/gradients_epoch_59_batch_2_channel_3.png', '../plots/grad_cam/gradients_epoch_41_batch_3_channel_3.png', '../plots/grad_cam/gradients_epoch_68_batch_1_channel_3.png', '../plots/grad_cam/gradients_epoch_48_batch_2_channel_3.png', '../plots/grad_cam/gradients_epoch_42_batch_3_channel_3.png', '../plo

In [10]:
mean_gradients

[0.9235979,
 0.9255037,
 0.9338146,
 0.9246568,
 0.9359551,
 0.935721,
 0.9344661,
 0.9386402,
 0.9214588,
 0.9061103,
 0.9125196,
 0.9405094,
 0.93246984,
 0.9317812,
 0.91904265,
 0.92688364,
 0.93688285,
 0.9294612,
 0.94125175,
 0.91137,
 0.92163324,
 0.93741494,
 0.93866396,
 0.9389044,
 0.925519,
 0.92163324,
 0.91137,
 0.9315962,
 0.9192326,
 0.94127357,
 0.92776555,
 0.922708,
 0.9243586,
 0.9185392,
 0.9185392,
 0.93117654,
 0.9345335,
 0.9311338,
 0.939584,
 0.9306531,
 0.9322586,
 0.92776555,
 0.93250096,
 0.93415976,
 0.9400193,
 0.939584,
 0.9323467,
 0.9394364,
 0.9290303,
 0.93386143,
 0.9405094,
 0.9375579,
 0.9255037,
 0.9227771,
 0.928354,
 0.928927,
 0.9125196,
 0.9173081,
 0.9394364,
 0.92686933,
 0.94216967,
 0.89876765,
 0.9306531,
 0.9214588,
 0.90473485,
 0.9201978,
 0.92854357,
 0.91229916,
 0.89876765,
 0.92442214,
 0.9250156,
 0.94125175,
 0.9246568,
 0.92853844,
 0.92776555,
 0.928354,
 0.91614205,
 0.91904265,
 0.9311338,
 0.9156411,
 0.9360635,
 0.93637866